In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net_v2(nn.Module):
    def __init__(self):
        super(Net_v2, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input 28x28x1, output 28x28x16, RF-3
        self.bnorm1 = nn.BatchNorm2d(16)
        
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) #input 28x28x16, output 26x28x32, RF-5
        self.bnorm2 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 16, 3) #input 28x28x32, output 26x26x16, RF-7
        self.bnorm3 = nn.BatchNorm2d(16)

        self.pool = nn.MaxPool2d(2, 2) # input 26x26x16, output 13x13x16 RF-8

        self.conv4 = nn.Conv2d(16, 16, 3) #input 13x13x16, output 11x11x16 
        self.bnorm4 = nn.BatchNorm2d(16)

        self.conv5 = nn.Conv2d(16, 16, 3) #input 11x11x16 output 9x9x16
        self.bnorm5 = nn.BatchNorm2d(16)

        self.conv6 = nn.Conv2d(16, 16, 3) # input 9x9x16 output 7x7x16
        self.bnorm6 = nn.BatchNorm2d(16)
        
        self.conv7 = nn.Conv2d(16, 10, 3) # input 7x7x16 output 5x5x10

        self.dropout = nn.Dropout(0.1)

    def forward(self, x):

        x = F.relu(self.bnorm1(self.dropout(self.conv1(x))))
        x = F.relu(self.bnorm2(self.dropout(self.conv2(x))))
        x = F.relu(self.bnorm3(self.dropout(self.conv3(x))))
        x = self.pool(x)
        x = F.relu(self.bnorm4(self.dropout(self.conv4(x))))
        x = F.relu(self.bnorm5(self.dropout(self.conv5(x))))
        x = F.relu(self.bnorm6(self.dropout(self.conv6(x))))
        x = self.conv7(x)
        x = F.avg_pool2d(x, 3)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.conv5 = nn.Conv2d(256, 512, 3)
#         self.conv6 = nn.Conv2d(512, 1024, 3)
#         self.conv7 = nn.Conv2d(1024, 10, 3)

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = F.relu(self.conv7(x))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [8]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net_v2().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
           Dropout-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 32, 28, 28]           4,640
           Dropout-5           [-1, 32, 28, 28]               0
       BatchNorm2d-6           [-1, 32, 28, 28]              64
            Conv2d-7           [-1, 16, 26, 26]           4,624
           Dropout-8           [-1, 16, 26, 26]               0
       BatchNorm2d-9           [-1, 16, 26, 26]              32
        MaxPool2d-10           [-1, 16, 13, 13]               0
           Conv2d-11           [-1, 16, 11, 11]           2,320
          Dropout-12           [-1, 16, 11, 11]               0
      BatchNorm2d-13           [-1, 16, 11, 11]              32
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Epoch:',epoch,'\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:

model = Net_v2().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader,epoch)

  0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 1 
Test set: Average loss: 0.0701, Accuracy: 9778/10000 (97.78%)



  0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 2 
Test set: Average loss: 0.0455, Accuracy: 9862/10000 (98.62%)



  0%|          | 0/469 [00:00<?, ?it/s]

...

Epoch: 18 
Test set: Average loss: 0.0228, Accuracy: 9936/10000 (99.36%)



  0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 19 
Test set: Average loss: 0.0179, Accuracy: 9946/10000 (99.46%)



loss=0.004373446106910706 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 36.02it/s]


Epoch: 20 
Test set: Average loss: 0.0207, Accuracy: 9937/10000 (99.37%)

